In [ ]:
# Chapter 02 - More PyMC

In [ ]:
# Typical imports
import arviz as az
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import theano.tensor as tt

## A little more PyMC3

### Model context

In PyMC3, we put all variables of interest in the context of a `Model`.

In [ ]:
with pm.Model() as model:
    parameter = pm.Exponential('poisson_param', lam=1.0)
    data_generator = pm.Poisson('data_generator', parameter)

We can continue to work within the context of a previously defined `Model` instance by using `with` and the name
of the previously defined variable referring to the `Model` instance.

In [ ]:
with model:
    data_plus_one = data_generator + 1

Even though we have defined variables in the context of a model, we can access these variables **outside** the
model context.

In [ ]:
parameter.tag.test_value

The first parameter to all model variables is its name. This name allows PyMC3 to access the variable name later.

Additionally, one can define a new variable with a previously used name in a different model.

In [ ]:
with pm.Model() as model:
    theta = pm.Exponential('theta', 2.0)
    data_generator = pm.Poisson('data_generator', theta)

We are free to name our model variables whatever we wish. Remember, reusing a model variable name results in
overwriting the previous model.

In [ ]:
with pm.Model() as ab_testing:
    p_A = pm.Uniform('P(A)', 0, 1)
    p_B = pm.Uniform('P(B)', 0, 1)

### PyMC3 Variables

All PyMC3 variables have an initial value (that is, a test value). Examining the variables previously defined:

In [ ]:
print(f'{parameter.tag.test_value=}')
print(f'{data_generator.tag.test_value=}')
print(f'{data_plus_one.tag.test_value=}')

The `test_value` is used only for the model to provide a starting point for sampling if no other starting point
is specified.

It will **not** change as a result of sampling; however, the initial state can be changed by specifying a value
for the `testval` parameter when creating the variable.

Specifying `testval` can be helpful if you are using a more unstable prior that may require a better starting
point for sampling.

In [ ]:
with pm.Model() as model:
    parameter = pm.Exponential('poisson_param', 1.0, testval=0.5)

In [ ]:
parameter.tag.test_value

In [ ]:
with pm.Model() as variable_model:
    # Stochastic variables have a name and other parameters specific to the distribution.
    # It is best to ues a description value for `name` since this value will be used to retrieve the
    # posterior distribution for subsequent analysis.
    some_variable = pm.DiscreteUniform('discrete_uniform_variable', 0, 4)

    # One can use the `shape` argument to model a multivariate (independent) stochastic variable.

    # Additionally, one can use the `shape` argument to handle the "annoying" case in which you wish
    # to model a sequence of values drawn from a single distribution that only differ by "index."
    # For example, instead of
    # beta_1 = pm.Uniform('beta_1', 0, 1)
    # beta_2 = pm.Uniform('beta_2', 0, 1)
    # ...
    #
    # One can wrap them in a single variable:
    N = 7
    betas = pm.Uniform('betas', 0, 1, shape=N)

    # The other category are deterministic variables. We can create a deterministic variable using the
    # `Deterministic` constructor.
    def some_function_of_other_variables(some_variable, betas):
        return some_variable + betas.mean()
    #
    # deterministic_variable = pm.Deterministic('deterministic variable', some_function_of_other_variables)
    # Note that the previous, commented out code generates an `AttributeError` about a `copy` attribute.
    # Based on other issues, I believe that the latest versions of PyMC3 no longer support `Deterministic`
    # as we used it above.

In [ ]:
# Using the `Deterministic` constructor is the most obvious way to create a deterministic variable; however,
# elementary operations like addition and exponentiation implicitly create deterministic variables.
# For example:
with pm.Model() as model:
    lambda_1 = pm.Exponential('lambda_1', 1.0)
    lambda_2 = pm.Exponential('lambda_2', 1.0)
    tau = pm.DiscreteUniform('tau', lower=0, upper=10)

new_deterministic_variable = lambda_1 + lambda_2

# Remember, to use a deterministic variable in a model, one must use the `Deterministic` constructor.

In [ ]:
# We can define a deterministic variable inside a model context:
n_data_points = 5  # We had 70 data points in chapter 01.
idx = np.arange(n_data_points)
with model:
    lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)

### Theano

In [ ]:
with pm.Model() as theano_test:
    p1 = pm.Uniform('p', 0, 1)
    p2 = 1 - p1
    p = tt.stack([p1, p2])

    assignment = pm.Categorical('assignment', p)

### Including observations in the model

In [ ]:
plt.figure(figsize=(12.5, 4))

samples = lambda_1.random(size=20000)
plt.hist(samples, bins=70, density=True, histtype='stepfilled')
plt.title('Prior distribution for $\lambda_1$')
plt.xlim(0, 8)

plt.show()

In [ ]:
data = np.array([10, 5])
with model:
    fixed_variable = pm.Poisson('fxd', 1, observed=data)
print(f'{fixed_variable.tag.test_value}')

In [ ]:
# Were using fake data here
data = np.array([10, 25, 15, 20, 35])
with model:
    obs = pm.Poisson('obs', lambda_, observed=data)
print(f'{obs.tag.test_value}')